# Sensitivity Analysis for DRK

**Author:** Agna Chan  
**Date:** December 2025  
**Affiliation:** Columbia University, Department of Statistics

---

This notebook analyzes how DRK results vary with:
1. Ambiguity radius ε
2. Confidence level α
3. Estimation window (lookback)
4. Sample size


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, "../src")

from kelly_robust.core.kelly import (
    drk_single_asset_closed_form, simulate_gbm_returns, simulate_wealth_paths
)
print("Imports complete.")

## 1. Sensitivity to Ambiguity Radius ε

We analyze how the optimal growth rate varies as we increase the ambiguity radius.

In [ ]:
# Parameters
TRUE_MU = 0.0005
TRUE_SIGMA = 0.02
TRUE_SIGMA2 = TRUE_SIGMA**2
HORIZON = 252
N_SIMS = 500

# Epsilon grid
epsilons = np.linspace(0, 0.002, 15)
growth_rates = []

print("Running sensitivity analysis for epsilon...")
for eps in epsilons:
    log_wealths = []
    np.random.seed(42)
    for _ in range(N_SIMS):
        hist = simulate_gbm_returns(TRUE_MU, TRUE_SIGMA, 252)
        fwd = simulate_gbm_returns(TRUE_MU, TRUE_SIGMA, HORIZON)
        
        mu_hat = np.mean(hist)
        sigma2_hat = np.var(hist, ddof=1)
        
        f = drk_single_asset_closed_form(mu_hat, sigma2_hat, eps, 0.0, 0.0, 1.0)
        wealth = simulate_wealth_paths(fwd, f)
        log_wealths.append(np.log(wealth[-1]))
    
    growth_rates.append(np.mean(log_wealths) / HORIZON)
    print(f"  ε = {eps:.4f}: Growth rate = {growth_rates[-1]*252*100:.2f}% ann.")

print("Done.")

In [ ]:
# Plot epsilon sensitivity
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(epsilons * 1000, np.array(growth_rates) * 252 * 100, "b-", linewidth=2, marker="o")

# Mark SE-based epsilon
se_epsilon = TRUE_SIGMA / np.sqrt(252) * 1.645
ax.axvline(se_epsilon * 1000, color="r", linestyle="--", linewidth=2, label=f"SE-based ε (90%)")

ax.set_xlabel("Ambiguity Radius ε (×10⁻³)", fontsize=12)
ax.set_ylabel("Annualized Growth Rate (%)", fontsize=12)
ax.set_title("DRK Performance vs Ambiguity Radius", fontsize=14)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig("../paper/figures/sensitivity_epsilon.pdf", bbox_inches="tight", dpi=150)
plt.show()
print("Figure saved.")

## 2. Sensitivity to Confidence Level α

In [ ]:
from kelly_robust.core.kelly import adaptive_conformal_kelly

alphas = np.linspace(0.01, 0.5, 15)
mean_fractions = []
mean_growths = []

print("Running sensitivity analysis for alpha...")
np.random.seed(42)

for alpha in alphas:
    fractions = []
    growths = []
    for _ in range(300):
        hist = simulate_gbm_returns(TRUE_MU, TRUE_SIGMA, 252)
        fwd = simulate_gbm_returns(TRUE_MU, TRUE_SIGMA, 252)
        
        ack = adaptive_conformal_kelly(hist, alpha=alpha)
        f = ack.fraction
        fractions.append(f)
        
        wealth = simulate_wealth_paths(fwd, f)
        growths.append(np.log(wealth[-1]) / 252)
    
    mean_fractions.append(np.mean(fractions))
    mean_growths.append(np.mean(growths))
    print(f"  α = {alpha:.2f}: Mean fraction = {mean_fractions[-1]:.4f}, Growth = {mean_growths[-1]*252*100:.2f}%")

print("Done.")

In [ ]:
# Plot alpha sensitivity
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Fraction vs alpha
ax1 = axes[0]
ax1.plot(alphas, mean_fractions, "g-o", linewidth=2)
ax1.set_xlabel("α (miscoverage rate)", fontsize=12)
ax1.set_ylabel("Mean Kelly Fraction", fontsize=12)
ax1.set_title("Kelly Fraction vs Confidence Level", fontsize=14)
ax1.grid(True, alpha=0.3)

# Right: Growth vs alpha
ax2 = axes[1]
ax2.plot(alphas, np.array(mean_growths) * 252 * 100, "b-o", linewidth=2)
ax2.set_xlabel("α (miscoverage rate)", fontsize=12)
ax2.set_ylabel("Annualized Growth Rate (%)", fontsize=12)
ax2.set_title("Growth Rate vs Confidence Level", fontsize=14)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig("../paper/figures/sensitivity_alpha.pdf", bbox_inches="tight", dpi=150)
plt.show()
print("Figure saved.")

## 3. Sensitivity to Sample Size

In [ ]:
sample_sizes = [50, 100, 150, 252, 504, 756, 1000]
mean_fractions_n = []
std_fractions_n = []

print("Running sensitivity analysis for sample size...")
np.random.seed(42)

for n in sample_sizes:
    fractions = []
    for _ in range(300):
        hist = simulate_gbm_returns(TRUE_MU, TRUE_SIGMA, n)
        ack = adaptive_conformal_kelly(hist, alpha=0.1)
        fractions.append(ack.fraction)
    
    mean_fractions_n.append(np.mean(fractions))
    std_fractions_n.append(np.std(fractions))
    print(f"  n = {n}: Mean fraction = {mean_fractions_n[-1]:.4f} ± {std_fractions_n[-1]:.4f}")

# Oracle fraction
f_oracle = TRUE_MU / TRUE_SIGMA2
print(f"
Oracle Kelly fraction: {f_oracle:.4f}")

In [ ]:
# Plot sample size sensitivity
fig, ax = plt.subplots(figsize=(10, 6))

ax.errorbar(sample_sizes, mean_fractions_n, yerr=std_fractions_n, 
            fmt="o-", color="purple", linewidth=2, capsize=5, markersize=8)
ax.axhline(y=f_oracle, color="green", linestyle="--", linewidth=2, label=f"Oracle f* = {f_oracle:.4f}")

ax.set_xlabel("Sample Size (n)", fontsize=12)
ax.set_ylabel("ACK Fraction", fontsize=12)
ax.set_title("Kelly Fraction Convergence vs Sample Size (α=0.10)", fontsize=14)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig("../paper/figures/sensitivity_sample_size.pdf", bbox_inches="tight", dpi=150)
plt.show()
print("Figure saved.")

## Conclusion

**Key Insights:**

1. **Epsilon sensitivity:** There exists an optimal ε that balances conservatism and aggressiveness
2. **Alpha sensitivity:** Lower α leads to more conservative (smaller) fractions but potentially lower growth
3. **Sample size:** ACK fraction converges to oracle as n → ∞, validating asymptotic consistency

The SE-based epsilon provides a good default choice, corresponding to ~90% confidence.